# connection to AWS Cloud

In [1]:
import pandas as pd
import numpy as np
import boto3
from sqlalchemy import create_engine
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
import os
from io import BytesIO
import ast

In [2]:

load_dotenv()

#credieantials for AWS
access_key=os.getenv('access_key')
secret_key=os.getenv('secret_key')

In [3]:

file_paths = {'bangalore':'data/bangalore_cars.xlsx','chennai': 'data/chennai_cars.xlsx', 'delhi':'data/delhi_cars.xlsx','hyderabad':'data/hyderabad_cars.xlsx','jaipur':'data/jaipur_cars.xlsx','kolkata':'data/kolkata_cars.xlsx'}


In [4]:
#pushing data to AWS
s3=boto3.client('s3',aws_access_key_id=access_key,aws_secret_access_key=secret_key)

In [5]:


for file,path in file_paths.items():
    s3.upload_file(path,'srvprojects','cardheko/'+file)

In [6]:
# AWS S3 bucket and file details
bucket_name = 'srvprojects'
keys = {
    'bangalore':'cardheko/bangalore',
    'chennai':'cardheko/chennai',
    'delhi':'cardheko/delhi',
    'hyderabad':'cardheko/hyderabad',
    'jaipur':'cardheko/jaipur',
    'kolkata':'cardheko/kolkata',
}
city_dataframes={}

for city, key in keys.items():
    try:
        # Fetch JSON file from S3
        response = s3.get_object(Bucket=bucket_name, Key=key)
        file_content = BytesIO(response['Body'].read())
        df = pd.read_excel(file_content, engine='openpyxl')
        df['City'] = city
        city_dataframes[city] = df
    except Exception as e:
        print(f"An error occurred while processing {key}: {e}")
# Combine all city DataFrames
if city_dataframes:
    combined_df = pd.concat(city_dataframes.values(), ignore_index=True)
    print(combined_df)
else:
    print("No data frames to combine.")

                                         new_car_detail  \
0     {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
1     {'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...   
2     {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
3     {'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...   
4     {'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...   
...                                                 ...   
8364  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
8365  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
8366  {'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...   
8367  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
8368  {'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...   

                                       new_car_overview  \
0     {'heading': 'Car overview', 'top': [{'key': 'R...   
1     {'heading': 'Car overview', 'top': [{'key': 'R...   
2     {'heading': 'Car overview', 'top': [{'key': 'R...   
3     {'heading': 'Car overview', 'top': [{'key': 'R...

In [7]:
#function to extract relevant fields 
def extract_features(row):
    try:
        car_details = ast.literal_eval(row['new_car_detail'])
        car_overview = ast.literal_eval(row['new_car_overview'])
        car_specs = ast.literal_eval(row['new_car_specs'])
    except (ValueError, SyntaxError):
        return pd.Series([None] * 7)
    
    fuel_type = car_details.get('ft', None)
    body_type = car_details.get('bt', None)
    transmission = car_details.get('transmission', None)
    km_driven = car_details.get('km', None)
    model_year = car_details.get('modelYear', None)
    No_owner=car_details.get('ownerNo', None)
    brand=car_details.get('oem',None)
    model=car_details.get('model',None)
    price=car_details.get('price',None)

    engine = None
    for spec in car_specs.get('top', []):
        if spec.get('key') == 'Engine':
            engine = spec.get('value', None)
            break

    Registration_Year=None
    for item in car_overview.get('top', []):
        if item.get('key') == 'Registration Year':
            Registration_Year = item.get('value', None)
            break

    return pd.Series([fuel_type, body_type, transmission, km_driven, model_year, price, engine, brand, model, Registration_Year],
                     index=['Fuel_Type', 'Body_Type', 'Transmission', 'KM_Driven', 'Model_Year', 'Price', 'Engine', 'Brand', 'Model', 'Registration_Year'])

extracted_data = combined_df.apply(extract_features, axis=1)

structured_df = pd.concat([extracted_data, combined_df['City']], axis=1)


print(structured_df)

# Convert to numeric by removing commas and converting to int/float
structured_df['KM_Driven'] = structured_df['KM_Driven'].replace({',': ''}, regex=True).astype(int)
structured_df['Engine'] = structured_df['Engine'].str.replace(' CC', '', regex=True).astype(float)

# Extract the year from the registration date
structured_df['Registration_Year'] = structured_df['Registration_Year'].str[-4:]
structured_df['Registration_Year'] = pd.to_numeric(structured_df['Registration_Year'], errors='coerce').astype('Int64')


def convert_price_to_lakh(price):
    price = price.replace('₹', '').replace(',', '').strip()
    
    # Split into amount and unit (Lakh, Crore, etc.)
    parts = price.split()
    if len(parts) == 2:
        amount = float(parts[0])
        unit = parts[1].lower()
        
        # Convert based on unit
        if unit == 'lakh':
            return amount
        elif unit == 'crore':
            return amount * 100
        elif unit == 'thousand':
            return amount / 100  
    return np.nan  

structured_df['Price'] = structured_df['Price'].apply(convert_price_to_lakh)

# Convert to numeric
structured_df['Price'] = pd.to_numeric(structured_df['Price'], errors='coerce')

print(structured_df)
print(structured_df.info())

# Fill missing values in 'Registration_Year' with values from 'Model_Year'
structured_df['Registration_Year'] = structured_df['Registration_Year'].fillna(structured_df['Model_Year'])
# Drop rows with missing values in the 'Engine' column
structured_df = structured_df.dropna(subset=['Engine'])

# Fill missing values in 'Price' with median price
median_price = structured_df['Price'].median()
structured_df['Price'] = structured_df['Price'].fillna(median_price)

# Count the number of duplicate rows across all columns
num_duplicates = structured_df.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

# Remove duplicate rows across all columns
structured_df = structured_df.drop_duplicates()

# Count the number of duplicate rows across all columns
num_duplicates = structured_df.duplicated().sum()

print(structured_df)
print(f"Number of duplicate rows: {num_duplicates}")
print(structured_df.info())
unique_values = structured_df['Brand'].unique()
print(f"Unique values in '{structured_df['Brand']}':")
print(unique_values)
unique_values = structured_df['Body_Type'].unique()
print(f"Unique values in '{structured_df['Body_Type']}':")
print(unique_values)

unique_values = structured_df['Fuel_Type'].unique()
print(f"Unique values in '{structured_df['Fuel_Type']}':")
print(unique_values)

# Save DataFrame to CSV file
structured_df.to_csv('cleaned_car_data.csv', index=False)

     Fuel_Type  Body_Type Transmission KM_Driven  Model_Year        Price  \
0       Petrol  Hatchback       Manual  1,20,000        2015     ₹ 4 Lakh   
1       Petrol        SUV       Manual    32,706        2018  ₹ 8.11 Lakh   
2       Petrol  Hatchback       Manual    11,949        2018  ₹ 5.85 Lakh   
3       Petrol      Sedan       Manual    17,794        2014  ₹ 4.62 Lakh   
4       Diesel        SUV       Manual    60,000        2015  ₹ 7.90 Lakh   
...        ...        ...          ...       ...         ...          ...   
8364    Petrol  Hatchback       Manual    10,000        2022  ₹ 5.10 Lakh   
8365    Petrol  Hatchback       Manual  1,20,000        2014  ₹ 1.80 Lakh   
8366    Petrol      Sedan    Automatic    50,000        2011  ₹ 5.50 Lakh   
8367    Petrol  Hatchback       Manual    40,000        2012  ₹ 1.40 Lakh   
8368    Diesel        SUV       Manual  1,20,000        2017     ₹ 5 Lakh   

       Engine          Brand                  Model Registration_Year  \
0 

In [8]:
#Establish DB connection AND create DATABASE

# Database connection details for the server (not the specific database)
server_config = {
    'host': 'myfinalproject.cls088i2o3dw.ap-south-1.rds.amazonaws.com',  # RDS endpoint or local host
    'port': 5432,
    'user': 'postgres',
    'password': 'admin123'
}

In [9]:


def create_database(db_name):
    conn = None
    cursor = None
    try:
        # Connect to the PostgreSQL server 
        conn = psycopg2.connect(**server_config)
        conn.autocommit = True  
        cursor = conn.cursor()

        # Create the database
        create_db_query = sql.SQL("CREATE DATABASE {db_name}").format(db_name=sql.Identifier(db_name))
        cursor.execute(create_db_query)
        print(f"Database '{db_name}' created successfully!")

    except psycopg2.Error as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the connection
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Usage
dbname='cardhekodb1'
create_database(dbname)

#pushing all csv to RDS

# Database connection details
db_config = {
    'host': 'myfinalproject.cls088i2o3dw.ap-south-1.rds.amazonaws.com', 
    'port': 5432,
    'user': 'postgres',  
    'password': 'admin123',  
    'database': dbname  
}

Database 'cardhekodb1' created successfully!


In [10]:
import pandas as pd
import psycopg2

def upload_csv_to_rds(csv_path, table_name, db_config):
    """
    Uploads a CSV file to a PostgreSQL RDS database table.

    Parameters:
        csv_path (str): Path to the CSV file.
        table_name (str): Name of the table to insert data into.
        db_config (dict): Database configuration with keys:
            - host
            - database
            - user
            - password
            - port
    """
    conn = None
    cursor = None
    try:
        # Read CSV into DataFrame
        df = pd.read_csv(csv_path)

        # Establish database connection
        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()

        # Define table schema based on the CSV structure
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {', '.join([f"{col} VARCHAR" for col in df.columns])}
        );
        """
        cursor.execute(create_table_query)

        # Prepare insert query
        placeholders = ", ".join(["%s"] * len(df.columns))
        insert_query = f"""
        INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({placeholders});
        """

        # Insert data row by row
        for _, row in df.iterrows():
            cursor.execute(insert_query, tuple(row))

        # Commit the transaction
        conn.commit()
        print(f"Data from {csv_path} uploaded to table {table_name} successfully.")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        # Ensure cursor and connection are closed properly
        if cursor:
            cursor.close()
        if conn:
            conn.close()


# Configuration and execution
csv_path = 'C:/Users/srvik/Desktop/github/Car Dheko - Used Car Price Prediction/cleaned_car_data.csv'
table_name = 'cleaned_car_data'


upload_csv_to_rds(csv_path, table_name, db_config)


Data from C:/Users/srvik/Desktop/github/Car Dheko - Used Car Price Prediction/cleaned_car_data.csv uploaded to table cleaned_car_data successfully.


In [11]:
def get_engine():
    return create_engine(
        f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
    )

# Fetch data from the database
def fetch_data(table_name):
    try:
        engine = get_engine()
        query = f"SELECT * FROM {table_name};"
        df = pd.read_sql(query, con=engine)
        return df
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
structured_df = fetch_data('cleaned_car_data')

In [12]:
structured_df

,fuel_type,body_type,transmission,km_driven,model_year,price,engine,brand,model,registration_year,city
0,Petrol,Hatchback,Manual,120000,2015,4.0,998.0,Maruti,Maruti Celerio,2015,bangalore
1,Petrol,SUV,Manual,32706,2018,8.11,1497.0,Ford,Ford Ecosport,2018,bangalore
2,Petrol,Hatchback,Manual,11949,2018,5.85,1199.0,Tata,Tata Tiago,2018,bangalore
3,Petrol,Sedan,Manual,17794,2014,4.62,1197.0,Hyundai,Hyundai Xcent,2014,bangalore
4,Diesel,SUV,Manual,60000,2015,7.9,1248.0,Maruti,Maruti SX4 S Cross,2015,bangalore
...,...,...,...,...,...,...,...,...,...,...,...
8217,Petrol,Hatchback,Manual,10000,2022,5.1,998.0,Maruti,Maruti Celerio,2022,kolkata
8218,Petrol,Hatchback,Manual,120000,2014,1.8,796.0,Maruti,Maruti Alto 800,2014,kolkata
8219,Petrol,Sedan,Automatic,50000,2011,5.5,1796.0,Mercedes-Benz,Mercedes-Benz C-Class,2011,kolkata
8220,Petrol,Hatchback,Manual,40000,2012,1.4,1197.0,Maruti,Maruti Ritz,2012,kolkata


In [13]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import pickle

In [14]:
#pre-processing and encoding
X=structured_df.drop(columns=['Price'])
Y=structured_df['Price']

# Define categorical and numerical columns
categorical_cols = ['Fuel_Type', 'Body_Type', 'Transmission', 'Brand', 'City']
numeric_cols = ['KM_Driven', 'Engine', 'Registration_Year', 'Model_Year']

# Create transformations
preprocessed=ColumnTransformer(transformers=[('num',StandardScaler(),numeric_cols),('cat',OneHotEncoder(),categorical_cols)])

X_preprocessed = preprocessed.fit_transform(X)

X_train,X_test,Y_train,Y_test=train_test_split(X_preprocessed,Y, test_size=0.2, random_state=69)

#train model

model={'Linear Regression':LinearRegression(),'Random Forest':RandomForestRegressor(random_state=69),'Decision Tree': DecisionTreeRegressor(random_state=69),
       'Gradient Boosting': GradientBoostingRegressor(random_state=69)}

for name,model in model.items():
    model.fit(X_train,Y_train)
    Y_pred=model.predict(X_test)
    print(f"{name} Evaluation:")
    print(f"MAE: {mean_absolute_error(Y_test, Y_pred):.2f}")
    print(f"MSE: {mean_squared_error(Y_test, Y_pred):.2f}")
    print(f"R2 score: {r2_score(Y_test, Y_pred):.2f}\n")

# Hyperparameter tuning for Random Forest using GridSearchCV
rf_params = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

grid_search_rf = GridSearchCV(RandomForestRegressor(random_state=42), rf_params, scoring='r2', cv=5, n_jobs=-1)
grid_search_rf.fit(X_train, Y_train)

print("Best parameters for Random Forest:", grid_search_rf.best_params_)
print("Best R2 score from Grid Search:", grid_search_rf.best_score_)

# Use the best estimator to predict on the test set
best_rf_model = grid_search_rf.best_estimator_
Y_pred_best_rf = best_rf_model.predict(X_test)

print("Optimized Random Forest Evaluation:")
print(f"MAE: {mean_absolute_error(Y_test, Y_pred_best_rf):.2f}")
print(f"MSE: {mean_squared_error(Y_test, Y_pred_best_rf):.2f}")
print(f"R2 Score: {r2_score(Y_test, Y_pred_best_rf):.2f}\n")

with open('best_rf_model.pkl', 'wb') as f:
    pickle.dump(best_rf_model, f)


with open('preprocessor.pkl', 'wb') as preprocessor_file:
    pickle.dump(preprocessed, preprocessor_file)

print("The optimized Random Forest model has been saved as 'best_rf_model.pkl'.")
print("The preprocessor has been saved as 'preprocessor.pkl'.")

KeyError: "['Price'] not found in axis"